In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('seleb_non_none.csv')
data.head()

,path,split,live_or_spoof,class_label
0,/Data/train/1/live/000184.jpg,train,live,0.0
1,/Data/train/1/live/032887.jpg,train,live,0.0
2,/Data/train/1/live/059151.jpg,train,live,0.0
3,/Data/train/1/live/072342.jpg,train,live,0.0
4,/Data/train/1/live/072393.jpg,train,live,0.0


In [3]:
data['path'] = data['path'].apply(lambda x: x.replace('/Data/', ''))
data.head()

,path,split,live_or_spoof,class_label
0,train/1/live/000184.jpg,train,live,0.0
1,train/1/live/032887.jpg,train,live,0.0
2,train/1/live/059151.jpg,train,live,0.0
3,train/1/live/072342.jpg,train,live,0.0
4,train/1/live/072393.jpg,train,live,0.0


In [4]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from timm import create_model
import torch.optim as optim
import torchvision.transforms as transforms

In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):

        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Формируем полный путь к изображению
        img_path = os.path.join(self.root_dir, self.dataframe.iloc[idx]['path'])
        
        # Проверяем существование файла
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Файл не найден: {img_path}")
        
        # Загружаем метки
        live_or_spoof = 1 if self.dataframe.iloc[idx]['live_or_spoof'] == 'live' else 0  
        class_label = self.dataframe.iloc[idx]['class_label']  
        
        # Открываем изображение
        image = Image.open(img_path).convert("RGB")
        
        # Применяем преобразования
        if self.transform:
            image = self.transform(image)
        
        return image, live_or_spoof, class_label

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Изменяем размер изображений
    transforms.ToTensor(),         # Преобразуем в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Нормализация
])

In [7]:
data_dir = "C:/Users/annam/Documents/Проекты/Диплом/Данные 2/CelebA_Spoof/Data"

train_df = data[data['path'].str.contains('train')]
test_df = data[data['path'].str.contains('test')]

In [8]:
# Создаем датасеты
train_dataset = CustomDataset(train_df, root_dir=data_dir, transform=transform)
test_dataset = CustomDataset(test_df, root_dir=data_dir, transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
model = create_model("mobilevitv2_175", pretrained=True)

in_features = model.head.in_features 
model.head = nn.Identity()  

model.global_pool = nn.AdaptiveAvgPool2d((1, 1))  

# два отдельных выхода
model.fc_live_or_spoof = nn.Linear(in_features, 2)  
model.fc_class_label = nn.Linear(in_features, 10) 

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [10]:
criterion_bin = nn.CrossEntropyLoss()  
criterion_multi = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
def train_model(model, dataloader, criterion_bin, criterion_multi, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, live_or_spoof, class_labels in dataloader:
            images = images.to(device)
            live_or_spoof = live_or_spoof.to(device)
            class_labels = class_labels.to(device)
            
            optimizer.zero_grad()
            
            features = model(images)  
            features = model.global_pool(features)  
            features = features.view(features.size(0), -1)  
            outputs_bin = model.fc_live_or_spoof(features)  
            outputs_multi = model.fc_class_label(features)  
            
            loss_bin = criterion_bin(outputs_bin, live_or_spoof)
            loss_multi = criterion_multi(outputs_multi, class_labels)
            loss = loss_bin + loss_multi
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Эпоха {epoch+1}/{num_epochs}, Потеря: {running_loss / len(dataloader)}")
    return model

# Обучение модели
model = train_model(model, train_loader, criterion_bin, criterion_multi, optimizer, num_epochs=10)


KeyboardInterrupt

